In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train= pd.read_csv('../input/bike-sharing-demand/train.csv')
df_test= pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
df_test.head()

**Feature Engineering**

In [ ]:
#First convert datetime column to datetime object
df_train['datetime']=pd.to_datetime(df_train['datetime'])

df_train['day']=df_train.datetime.apply(lambda x:x.day)
df_train['month']=df_train.datetime.apply(lambda x:x.month)
df_train['year']=df_train.datetime.apply(lambda x:x.year)
df_train['hour']=df_train.datetime.apply(lambda x:x.hour)


df_train=df_train.drop('datetime',axis=1)
df_train.head()

In [ ]:
means = df_train.groupby(['day'])['count'].mean().to_dict()
# df_train['day']=df_train['day'].map(means)

means1 = df_train.groupby(['month'])['count'].mean().to_dict()
df_train['month']=df_train['month'].map(means1)

means2 = df_train.groupby(['year'])['count'].mean().to_dict()
df_train['year']=df_train['year'].map(means2)

means3 = df_train.groupby(['hour'])['count'].mean().to_dict()
df_train['hour']=df_train['hour'].map(means3)


In [ ]:
# Feature Engineering for test set
df_test['datetime']=pd.to_datetime(df_test['datetime'])

df_test['day']=df_test.datetime.apply(lambda x:x.day)
df_test['month']=df_test.datetime.apply(lambda x:x.month)
df_test['year']=df_test.datetime.apply(lambda x:x.year)
df_test['hour']=df_test.datetime.apply(lambda x:x.hour)

date=df_test['datetime']
df_test=df_test.drop('datetime',axis=1)
df_test.head()


In [ ]:
# df_test['day']=df_test['day'].map(means)
df_test['month']=df_test['month'].map(means1)
df_test['year']=df_test['year'].map(means2)
df_test['hour']=df_test['hour'].map(means3)


In [ ]:
df_test.head()

In [ ]:
# # convert some variables from int or float types to categorical type to visualize it 
# cat_list=["hour","day","month","season","weather","holiday","workingday",'year']
# for variable in cat_list:
#     df_train[variable]=df_train[variable].astype('category')

In [ ]:
df_train.dtypes

In [ ]:
# Some EDAs
corrmatrix = df_train.corr()
f, ax = plt.subplots(figsize=(30,10))
mask = np.array(corrmatrix)
mask[np.tril_indices_from(mask)] = False
sns.heatmap(corrmatrix,mask=mask, vmax=.9, square=True,annot=True,cbar=True)


In [ ]:
# f,ax= plt.subplots(nrows=1,ncols=2)
# f.set_size_inches(10,5)
# sns.boxplot(data=df_train,y='count',ax=[0][0])
# sns.boxplot(data=df_train,x='season',y='count',ax=[0][1])

In [ ]:
df_final=df_train.copy()
df_final['count']=np.log(df_final['count']+1)
print(df_final.head())
Y_train = df_final["count"]
X_train = df_final.drop(['count','registered','casual'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X_train , Y_train,  train_size=0.8, test_size=0.2, random_state =0)

In [ ]:
#n_estimators=300
model=RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
preds=model.predict(x_test)
score=np.sqrt(mean_squared_error(y_test,preds))

In [ ]:
print(score)
print(np.sqrt(mean_squared_error(y_train,model.predict(x_train))))

In [ ]:
df_test.head()

In [ ]:
fi=model.predict(df_test)
fi=np.round(np.exp(fi)-1)

In [ ]:
output = pd.DataFrame({'datetime': date,
                       'count': fi})
output.to_csv('submission.csv', index=False)